In [1]:
import numpy as np
import pandas as pd


In [2]:
data_train = pd.read_csv('../data_cleaned/data_train.csv')
item_categories = pd.read_csv('../data_cleaned/item_categories.csv')
items = pd.read_csv('../data_cleaned/items.csv')
shops = pd.read_csv('../data_cleaned/shops.csv')

In [3]:
#merged = data_train.merge(items, how='left').merge(item_categories, how='left').merge(shops, how='left')

In [4]:
def parse_city(shop_name):
    if shop_name.split()[0] == '!Якутск':
        return  'Якутск'

    if shop_name.split()[0] == 'Сергиев':
            return  'Сергиев Посад'
    else:
        return shop_name.split()[0]

In [5]:
#dates = pd.to_datetime(data_train['date'], format='%d.%m.%Y')


In [6]:
grouped=data_train.groupby(['shop_id','item_id','date_block_num']).agg({'item_price':'mean',
                                                                    'item_cnt_day':'sum'
                                                                    })#take some (a lot)))) ) time

In [7]:
grouped

item_price  item_cnt_day
shop_id item_id date_block_num                          
2       27      0                   2499.0           1.0
                17                   498.0           1.0
        30      2                    359.0           1.0
                5                    399.0           1.0
                15                   169.0           1.0
...                                    ...           ...
59      22164   27                   699.0           2.0
                30                   699.0           1.0
        22167   9                    299.0           1.0
                11                   299.0           2.0
                17                   299.0           1.0

[1608225 rows x 2 columns]

In [8]:
grouped=grouped.reset_index()

In [9]:
grouped

,shop_id,item_id,date_block_num,item_price,item_cnt_day
0,2,27,0,2499.0,1.0
1,2,27,17,498.0,1.0
2,2,30,2,359.0,1.0
3,2,30,5,399.0,1.0
4,2,30,15,169.0,1.0
...,...,...,...,...,...
1608220,59,22164,27,699.0,2.0
1608221,59,22164,30,699.0,1.0
1608222,59,22167,9,299.0,1.0
1608223,59,22167,11,299.0,2.0


In [10]:
merged = grouped.merge(items, how='left').merge(item_categories, how='left').merge(shops, how='left')

In [11]:
merged.isna().sum()

shop_id               0
item_id               0
date_block_num        0
item_price            0
item_cnt_day          0
item_name             0
item_category_id      0
item_category_name    0
shop_name             0
dtype: int64

In [12]:
import calendar

def number_to_month(numbers):
    # Create a list of month names using the calendar module
    month_names = list(calendar.month_name)[1:]  # Extract months from 1 to 12 (exclude the empty element at index 0)
    
    # Convert the numbers to month names
    result = [month_names[num % 12] for num in numbers]  # Use num % 12 to handle numbers beyond 11
    
    return result

# Example usage:
numbers = [0, 1, 2, 11, 12, 33]
print(number_to_month(numbers))

['January', 'February', 'March', 'December', 'January', 'October']


In [13]:
def convert_to_year(month_num):
    # January 2013 is the starting point
    start_year = 2013
    
    # Calculate the year
    year = start_year + (month_num // 12)
    
    return year

In [14]:
merged['month'] = number_to_month(merged['date_block_num'])
merged['year'] = merged['date_block_num'].apply(convert_to_year)

In [15]:
merged['super_category'] = merged['item_category_name'].apply(lambda a: a.split()[0])

In [16]:
#merged = merged.drop(['shop_id','item_id','item_category_id'],axis=1)

In [17]:
merged=merged.rename(columns={'item_price':'avg_item_price','item_cnt_day':'item_cnt_month'})

In [18]:
item_lags=list([*range(1,7),12,13,24])
price_lags=list([*range(1,7),12,24])

In [19]:

def create_lags(data,item_lags=None,price_lags=None):
    lagged_items = data[['date_block_num','item_cnt_month','shop_id','item_id']].copy()
    for lag in item_lags:
        lagged_items['date_block_num']-=lag#previous month becomes present
        data=data.merge(lagged_items, how='left',on=['date_block_num','shop_id','item_id'], suffixes=('', f'_lag_{lag}'))
        lagged_items['date_block_num']+=lag

    lagged_prices = data[['date_block_num','avg_item_price','shop_id','item_id']].copy()
    for lag in price_lags:
        lagged_prices['date_block_num']-=lag#previous month becomes present
        data=data.merge(lagged_prices, how='left',on=['date_block_num','shop_id','item_id'], suffixes=('', f'_lag_{lag}'))
        lagged_prices['date_block_num']+=lag

        
    return data

In [20]:
lagged = create_lags(merged,item_lags=item_lags,price_lags=price_lags)#takes a bit time

In [21]:
lagged.isna().sum()

shop_id                        0
item_id                        0
date_block_num                 0
avg_item_price                 0
item_cnt_month                 0
item_name                      0
item_category_id               0
item_category_name             0
shop_name                      0
month                          0
year                           0
super_category                 0
item_cnt_month_lag_1      930003
item_cnt_month_lag_2     1014930
item_cnt_month_lag_3     1087510
item_cnt_month_lag_4     1142962
item_cnt_month_lag_5     1189600
item_cnt_month_lag_6     1229430
item_cnt_month_lag_12    1411072
item_cnt_month_lag_13    1434550
item_cnt_month_lag_24    1567487
avg_item_price_lag_1      930003
avg_item_price_lag_2     1014930
avg_item_price_lag_3     1087510
avg_item_price_lag_4     1142962
avg_item_price_lag_5     1189600
avg_item_price_lag_6     1229430
avg_item_price_lag_12    1411072
avg_item_price_lag_24    1567487
dtype: int64

In [22]:
lagged = lagged.drop(['avg_item_price'],axis=1)

In [23]:
lagged

,shop_id,item_id,date_block_num,item_cnt_month,item_name,item_category_id,item_category_name,shop_name,month,year,...,item_cnt_month_lag_13,item_cnt_month_lag_24,avg_item_price_lag_1,avg_item_price_lag_2,avg_item_price_lag_3,avg_item_price_lag_4,avg_item_price_lag_5,avg_item_price_lag_6,avg_item_price_lag_12,avg_item_price_lag_24
0,2,27,0,1.0,"007 Legends [PS3, русская версия]",19,Игры - PS3,"Адыгея ТЦ ""Мега""",January,2013,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,27,17,1.0,"007 Legends [PS3, русская версия]",19,Игры - PS3,"Адыгея ТЦ ""Мега""",June,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,30,2,1.0,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,Кино - DVD,"Адыгея ТЦ ""Мега""",March,2013,...,1.0,NaN,NaN,NaN,399.0,NaN,NaN,NaN,NaN,NaN
3,2,30,5,1.0,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,Кино - DVD,"Адыгея ТЦ ""Мега""",June,2013,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,30,15,1.0,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,Кино - DVD,"Адыгея ТЦ ""Мега""",April,2014,...,NaN,NaN,169.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1608220,59,22164,27,2.0,ЯРОСТЬ (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",April,2015,...,NaN,NaN,NaN,NaN,699.0,NaN,NaN,NaN,NaN,NaN
1608221,59,22164,30,1.0,ЯРОСТЬ (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",July,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1608222,59,22167,9,1.0,Язык запросов 1С:Предприятия 8 (+CD). Хрустале...,49,Книги - Методические материалы 1С,"Ярославль ТЦ ""Альтаир""",October,2013,...,NaN,NaN,NaN,299.0,NaN,NaN,NaN,NaN,NaN,NaN
1608223,59,22167,11,2.0,Язык запросов 1С:Предприятия 8 (+CD). Хрустале...,49,Книги - Методические материалы 1С,"Ярославль ТЦ ""Альтаир""",December,2013,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,299.0,NaN,NaN


In [24]:
!pwd

/home/valeriy/python_projects/predict_future_sales/EDA
